# Image Quality Metrics

In image processing and computer vision, evaluating the quality of images after transformations (such as resizing or super-resolution) is crucial. Several metrics are commonly used to assess image quality, including PSNR (Peak Signal-to-Noise Ratio) and SSIM (Structural Similarity Index). These metrics help quantify the similarity between the original and processed images.

## PSNR (Peak Signal-to-Noise Ratio)

**PSNR** is a widely used metric for measuring the quality of reconstructed images. It compares the maximum possible pixel value of the image with the noise or error introduced during image processing.

### Mathematical Explanation

1. **Definition:**
   - PSNR is calculated using the Mean Squared Error (MSE) between the original image \( I \) and the processed image \( \hat{I} \):
     $$
     \text{MSE} = \frac{1}{H \times W} \sum_{i=1}^{H} \sum_{j=1}^{W} (I(i, j) - \hat{I}(i, j))^2
     $$
   - Where \( H \) and \( W \) are the height and width of the image.

2. **Peak Signal-to-Noise Ratio Calculation:**
   - PSNR is computed from the MSE using the maximum pixel value \( M \) (for 8-bit images, \( M = 255 \)):
     $$
     \text{PSNR} = 10 \cdot \log_{10}\left(\frac{M^2}{\text{MSE}}\right)
     $$

### Practical Considerations

1. **Interpretation:**
   - Higher PSNR values indicate better image quality, with fewer differences between the original and processed images.
   - Typical PSNR values range from 20 dB to 50 dB, where values above 30 dB are generally considered good.

2. **Limitations:**
   - PSNR does not always correlate with perceived visual quality and may not reflect structural changes in the image.

## SSIM (Structural Similarity Index)

**SSIM** is a metric that assesses the similarity between two images based on luminance, contrast, and structure. Unlike PSNR, SSIM considers changes in the image structure and is more aligned with human visual perception.

### Mathematical Explanation

1. **Definition:**
   - SSIM is computed using local windows and evaluates the following components:
     - Luminance \( l(x, y) \)
     - Contrast \( c(x, y) \)
     - Structure \( s(x, y) \)

2. **SSIM Formula:**
   - The SSIM index between two images \( I \) and \( \hat{I} \) is given by:
     $$
     \text{SSIM}(x, y) = \frac{(2 \mu_I \mu_{\hat{I}} + C_1) \cdot (2 \sigma_{I \hat{I}} + C_2)}{(\mu_I^2 + \mu_{\hat{I}}^2 + C_1) \cdot (\sigma_I^2 + \sigma_{\hat{I}}^2 + C_2)}
     $$
   - Where \( \mu_I \) and \( \mu_{\hat{I}} \) are the mean values of \( I \) and \( \hat{I} \), \( \sigma_I^2 \) and \( \sigma_{\hat{I}}^2 \) are their variances, and \( \sigma_{I \hat{I}} \) is their covariance. \( C_1 \) and \( C_2 \) are constants to avoid instability.

### Practical Considerations

1. **Interpretation:**
   - SSIM values range from -1 to 1, where 1 indicates perfect similarity.
   - Higher SSIM values correspond to better structural similarity and visual quality.

2. **Limitations:**
   - SSIM may be sensitive to the choice of parameters and window size.

## Example Code

Here is a Python code snippet to compute PSNR and SSIM using the `scikit-image` library:

**Importing libraries**

In [3]:
from skimage.metrics import peak_signal_noise_ratio as psnr, structural_similarity as ssim
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2

**Show an example**

In [ ]:
# Load images
image_hr = Image.open('./images/image_hr.png')
image_lr_2 = Image.open('./images/image_lr_2.png')
image_lr_4 = Image.open('./images/image_lr_4.png')


def upsample(image, scale_factor, method):
    if   method == "BILINEAR": method_up = Image.BILINEAR
    elif method == "BICUBIC":  method_up = Image.BICUBIC
    elif method == "NEAREST":  method_up = Image.NEAREST
    elif method == "LANCZOS":  method_up = Image.LANCZOS
    else:
        print(method, "not found, using bicubic")
        method_up = Image.BICUBIC

    # Resize the image using bicubic interpolation
    return image.resize(
        (int(image.width * scale_factor), int(image.height * scale_factor)), 
        method_up
    )

# Load an image
image_hr   = Image.open('./images/image_hr.png')
image_lr_2 = Image.open('./images/image_lr_2.png')
image_lr_4 = Image.open('./images/image_lr_4.png')


# Create a figure with subplots
fig, axs = plt.subplots(2, 4, figsize=(15, 8), gridspec_kw={'width_ratios': [1, 1, 1, 1]})


for idx, method in enumerate(["NEAREST", "BILINEAR", "BICUBIC", "LANCZOS"]):
    image_up = upsample(image_lr_2, scale_factor=2, method=method)
    psnr_value = psnr(np.array(image_hr), np.array(image_up))
    ssim_value = ssim(np.array(image_hr), np.array(image_up), win_size=3, channel_axis=-1)  # Set win_size=3 to ensure it works for small images
    axs[0, idx].imshow(image_up)
    axs[0, idx].set_title(f'{method} x2\nPSNR: {psnr_value:.2f}, SSIM: {ssim_value:.2f}')
    axs[0, idx].axis('off')

    image_up = upsample(image_lr_4, scale_factor=4, method=method)
    psnr_value = np.round(psnr(np.array(image_hr), np.array(image_up)), 2)
    ssim_value = np.round(ssim(np.array(image_hr), np.array(image_up), win_size=3, channel_axis=-1), 4)  # Set win_size=3 to ensure it works for small images
    
    axs[1, idx].imshow(image_up)
    axs[1, idx].set_title(f'{method} x4\nPSNR: {psnr_value:.2f}, SSIM: {ssim_value:.2f}')
    axs[1, idx].axis('off')

To visualize the differences between the super-resolved (or interpolated) images and the original image, you can use several techniques, such as:

- **Absolute Difference Image**: Subtract the original image from the super-resolved image pixel by pixel and display the absolute difference. This will highlight areas where there are greater discrepancies.

- **Residual Image**: Similar to the difference image, but without taking the absolute value. This can show areas that are overestimated or underestimated.

Below is an example of how to implement and visualize the absolute difference image in your code. This will highlight the most significant differences between each super-resolved image and the original.

In [5]:
# Define function to calculate absolute difference
def absolute_difference(image1, image2):
    image1 = np.array(image1).astype("float32")
    image2 = np.array(image2).astype("float32")
    return np.abs(image1 - image2)

# Define function to calculate residual
def residual(image1, image2):
    image1 = np.array(image1).astype("float32")
    image2 = np.array(image2).astype("float32")
    a = image1 - image2
    return a

# Define function to calculate heatmap
def prepare_heatmap(heatmap, scaleHM=False):
    # Ensure the difference image is in the range [0, 255] and convert to uint8
    if scaleHM:
        if heatmap.max() == heatmap.min(): # images are same
            pass
        else:
            heatmap = (heatmap - heatmap.min()) / (heatmap.max() - heatmap.min()) * 255
    heatmap = np.clip(heatmap, 0, 255).astype(np.uint8)
    # Convert to grayscale if needed
    if len(heatmap.shape) == 3:
        heatmap = cv2.cvtColor(heatmap, cv2.COLOR_RGB2GRAY)
    # Apply color map
    return heatmap

In [ ]:
def upsample(image, scale_factor, method):
    if   method == "BILINEAR": method_up = Image.BILINEAR
    elif method == "BICUBIC":  method_up = Image.BICUBIC
    elif method == "NEAREST":  method_up = Image.NEAREST
    elif method == "LANCZOS":  method_up = Image.LANCZOS
    else:
        print(method, "not found, using bicubic")
        method_up = Image.BICUBIC

    # Resize the image using bicubic interpolation
    return image.resize(
        (int(image.width * scale_factor), int(image.height * scale_factor)), 
        method_up
    )

# Load an image
image_hr   = Image.open('./images/image_hr.png').convert('RGB')
image_lr_2 = Image.open('./images/image_lr_2.png').convert('RGB')
image_lr_4 = Image.open('./images/image_lr_4.png').convert('RGB')

# Create a figure with subplots
fig, axs = plt.subplots(3, 5, figsize=(20, 10), gridspec_kw={'width_ratios': [1, 1, 1, 1, 1]})

for idx, method in enumerate(["ORIGINAL", "NEAREST", "BILINEAR", "BICUBIC", "LANCZOS"]):
    if method == "ORIGINAL":
        image_up = image_hr
    else:
        image_up = upsample(image_lr_2, scale_factor=2, method=method)

    psnr_value = psnr(np.array(image_hr), np.array(image_up))
    ssim_value = ssim(np.array(image_hr), np.array(image_up), win_size=3, channel_axis=-1)
    abs_diff = absolute_difference(image_hr, image_up)
    residual_img = residual(image_up, image_hr)

    axs[0, idx].imshow(image_up)
    axs[0, idx].set_title(f'{method} x2\nPSNR: {psnr_value:.2f}, SSIM: {ssim_value:.2f}')
    axs[0, idx].axis('off')
    
    axs[1, idx].imshow(prepare_heatmap(abs_diff), cmap='gray', vmin=0, vmax=255)
    axs[1, idx].set_title(f"Abs. Diff.")
    axs[1, idx].axis('off')

    axs[2, idx].imshow(prepare_heatmap(residual_img, scaleHM=True), cmap='gray', vmin=0, vmax=255)
    axs[2, idx].set_title(f"Residual Diff.")
    axs[2, idx].axis('off')

plt.tight_layout()
plt.show()